# Creating languages with the Meta Attack Language
This notebook will demonstrate how to build a simple domain-specific language using the Meta Attack Language. Note that this work does not translate into Jupyter Notebooks without significant effort at this point in time, putting it firmly in the "not usable" camp of FDD3001. Therefore, this will be a non-executable tutorial for the sake of the exercise. The examples can be followed and executed, provided one sets up the required environment.

## Objectives
1. Demonstrate the fundamentals of the Meta Attack Language.
2. Construct a simple domain-specific language.
3. Simulate attacks using the newly constructed language.

## Key references
- Johnson, P., Lagerström, R., Ekstedt, M., 2018. A meta language
for threat modeling and attack simulations, in: Proceedings of
the 13th International Conference on Availability, Reliability and
Security, pp. 1–8.
- MAL Syntax. https://github.com/mal-lang/mal-documentation/wiki/MAL-Syntax
- toasterLang. https://github.com/viktorengstrom/toasterLang/

## Setting up the Meta Attack Language environment
The simplest way to work with the Meta Attack Language is to follow the exampleLang instructions at https://github.com/mal-lang/exampleLang/blob/master/README.md

Note that the environment is developed to run on Ubuntu specifically. If you are not already running a Linux distribution, the easiest solution is to run Ubuntu on a virtual machine, e.g, using VirtualBox.
Easiest might be to download and import this prepared Ubuntu server virtual machine: https://kth.box.com/s/64w06fwggv4b3eyfxjwd969l1gv1t55r

The login is fddrepro:fddrepro and the project can navigated to with 'cd \~/toasterLang/toasterLang'. The source file can be viewed or edited, for example, less, vim or nano at '\~/toasterLang/toasterLang/src/main/mal/toasterLang.mal'. The test file is at '\~/toasterLang/toasterLang/src/test/mal/java/org/toasterlang/viktorengstrom/TestToaster.java'. For example, the following allows one to view and scroll through the MAL source code 'less \~/toasterLang/toasterLang/src/main/mal/toasterLang.mal'

## Key concepts

The Meta Attack Language (MAL) is a high-level programmatic specification langauge. It is used to create custom threat modeling languages. These are capable of describing a system and reason about its security. There are two corresponding modles to keep in mind

- The descriptive structural model.
- The attack graph generated from the structural model.

The structural model describes a system in question. This is done by defining assets and associations between assets. That is, things and relationships between things.

The attack graph is constructed from pre-defined attack and defense logic embedded into the structural model. Therefore, creating a structural model implicitly constructs the graph as well. However, the user only needs knowledge of the structural descriptive side of things. The resulting graph will essentially be a map of all possible steps in an attack, according to the custom modeling language. Following the paths via Monte Carlo simulations extracts the more interesting attack paths.

# Designing and building a toaster language

To demonstrate MAL without going too deep into specifics, this guide will construct a modeling language for an ordinary single slot kitchen toaster. The "attacks" in question will be to either steal toast or add your own toast. This toy example will highlight the following
- Basic MAL syntax and concepts.
- How to construct and design custom modeling languages using MAL.
- How to encode attack and defense logic.
- How to instantiate models and simulate attacks.

## Preamble

The MAL code will be written in a simple textfile called *toasterLang.mal*. In the exampleLang project structure the file will be placed under "\*/FDD_toasterLang/src/main/mal".

The MAL compiler requires some preamble before the toast langauge can be compiled. Add the following the the empty .mal file

> #id: "org.toasterLang"

> #version: "1.0.0"

> category System {}

ID and version are project descriptions. Categories structure MAL specifications but generally have little functional purposes. However, the compiler expects at least one category. Thus, our code will be written inside the system category.

## The necessary toaster components

The goal of the exercise is to design a modeling language capable of simulating the following situations
1. Unauthorized removal of bread from a toaster.
2. Unauthorized addition of break to a toaster.

The first step in the design process is studying the system itself. For the purposes of this exercise, the following components are necesary

- Toaster - the system itself.
- Slot - the toast slot on top of the toaster frame.
- Toast - bread slices inserted or removed into toast slots.
- Owner - The agent affected by the toaster outputs. Common in security, but mainly used for demonstration purposes here.

The second step is to clarify the relationships between the components:

- Toaster -- contains --> Slot
- Owner -- owns --> Toaster
- Slot -- contains --> Toast

Both the components and relationships outlined thus far will now be translated into MAL in a 1:1 fashion.


## Translating components to MAL

Components are represented in MAL as assets. The asset declaration syntax is

> asset <name> <extends> {...}

For the time being, the four components defined can be translated directly into MAL as such

> asset Toaster {}
    
> asset Slot {}
    
> asset Toast {} 
    
> asset Owner {}



## Translating component relationships to MAL

Component relationships are defined in MAL as associations. These are contained in their own "asset" of sorts, defined at the end of a MAL specification file as such

> associations {...}

Associations and defined in-between the curly-braces of the associations definition. The syntax of association entries is

> Asset1 [role1] <multiplicity1> <-- AssociationName --> <multiplicity2> [role2] Asset2
    
Associations are bidirectional relationships between two assets (Asset1 and Asset2). The multiplicities can be one of
- 1 - The asset can only be connected to 1 instance of the corresponding asset.
- 1..* - The asset in question can be connected to 1 or more instances of the corresponding asset.
- \* - The asset in question can be connected to any amount of instances of the corresponding asset.

The AssociationName is has no function and serves only as documentation for developers.
    
The role names in brackets will be revisited when attacks and defenses are added.
    
Translating the three relationships added into MAL will result in the following lines
    
> associations{
   
>  Owner [owner] 1 <-- owns --> * [toasters] Toaster 
    
>  Toaster [toaster] 1 <-- hosts --> 1 [slot] Slot 
    
>  Slot [slot] 1 <-- toasts --> 0..1 [toast] Toast
    
>}
    
The entire MAL specification file so far should contain the following sequence of lines
    
> #id: "org.toasterLang"

> #version: "1.0.0"
    
> category System {
    
> asset Toaster {}
    
> asset Slot {}
    
> asset Toast {} 
    
> asset Owner {}

> }
    
> associations{
   
>  Owner [owner] 1 <-- owns --> * [toasters] Toaster 
    
>  Toaster [toaster] 1 <-- hosts --> 1 [slot] Slot 
    
>  Slot [slot] 1 <-- toasts --> 0..1 [toast] Toast
    
> }
    

## Stealing toast

The two types of attack steps must be introduced to start modeling attacks. The basic syntax for a generic attack step is

> \<type\> name @tag [ProbabilityDistribution(p)]

The tags and probability distribution will not be covered in this tutorial.

The type can be either an OR (|) or an AND (&). OR steps require at least one preceding step in an attack sequence to be completed. In contrast, the AND-step requires all preceding attack steps to be completed.

First off, stealing toast means removing it. Therefore, a remove step can be defined for the Toast asset. Removing toast requires the attacker to reach it in any number of ways, making it an OR-step. The Toast asset should, therefore, be revised into the following

> asset Toast {

> | remove

>}

If an attacker reaches remove, they have succeeded. However, removing toast is a consequences of the theft itself, which is an attack against the Toaster. In other words, stealing from the toaster leads to toast being removed.

Going backwards, toast removal is caused by an event involving the toaster. Namely, a successful removal attempt. The removal attempt is added to the Toaster asset as such

> asset Toaster{

> | attemptToastRemoval

> -> toastRemoval

>}

What has now been introduced is the leads to (->) operators. This signifies what should happen next in the chain of events, should the corresponding attack succeed.

Before writing the toastRemoval step, however, another condition is required. Removing toast realistically requires toast to be present in the toaster. Therefore, an existence (E) check is also required

> asset Toaster{

> | attemptToastRemoval

> -> toastRemoval

> E ifToastExists

> <- slot.toast

> -> toastRemoval

>}

Two important operators were introduced: requires (<-) and collect (a.b). Requires denotes which associations to check during existence and non-existence (!E) checks. The collect operators references other assets and attack steps. Recall the associations for Toaster and Slot

>  Toaster [toaster] 1 <-- hosts --> 1 [slot] Slot 
    
>  Slot [slot] 1 <-- toasts --> 0..1 [toast] Toast

When writing attack, or defense, logic for the Toaster asset, only the nearest asset and the toaster itself is referable. Referencing steps within the toaster itself only requires the attack step name, e.g., toastRemoval. Referencing an external asset or attack step requires the collect operator (.), e.g., slots.toast. The names used are defined by the \[bracketed\] names in the associations. Therefore, referencing a Slot from Toaster becomes

> slot.toast

Referencing the Toaster backwards from the Toast conversely becomes

> slot.toaster

The sequences are given by following the association definitions forwards and backwards.

The final step is to add the toastRemoval step. Its success criteria are a removal attempt and the existence of a Toast asset. As a rule, existence checks operate separately from the attack step sequences. IN other words, the sequence is not *attemptRemoval -> ifToastExists -> toastRemoval*. The removal attempt happens parallel to the existence. Therefore, the final step becomes an AND-step as such

> asset Toaster{

> | attemptToastRemoval

> -> toastRemoval

> E ifToastExists

> <- slots.toast

> -> toastRemoval

> & toastRemoval

> -> slots.toast.remove

>}

This completes the first sequence of events. The full file so far should be

> #id: "org.toasterLang"

> #version: "1.0.0"

> category System {

> asset Toaster{

> | attemptToastRemoval

> -> toastRemoval

> E ifToastExists

> <- slots.toast

> -> toastRemoval

> & toastRemoval

> -> slots.toast.remove

>}

> > asset Slot {}

> asset Toast {

> | remove

>}

> asset Owner {}

>}

> associations{
   
>  Owner [owner] 1 <-- owns --> * [toasters] Toaster 
    
>  Toaster [toaster] 1 <-- hosts --> * [slots] Slot 
    
>  Slot [slot] 1 <-- toasts --> 0..1 [toast] Toast
    
>}




## Adding toast

This attack largely mirrors stealing toast. To add toast to the toaster, an attempt should be made as before

> asset Toaster{

> | attemptToasting

> -> unauthorizedToasting

> }

This attack, however, depends on the absence of toast in the toaster. Therefore, a corresponding non-existence (!E) check is performed

> asset Toaster{

> | attemptToasting

> -> unauthorizedToasting

> !E ifToastNotExists

> <- slots.toast

> -> unauthorizedToasting

> }

Finally, the unauthorized toasting can be written as dependent on both events above

> asset Toaster{

> | attemptToasting

> -> unauthorizedToasting

> !E ifToastNotExists

> <- slots.toast

> -> unauthorizedToasting

> & unauthorizedToasting

> -> owner.hacked

> }

The unauthorized toasting does not affect an existing Toast asset like the theft. Additionally, MAL simulation models are static in the sense that they cannot be updated at runtime. Therefore, adding a toast asset as a consequence of unauthorizedToasting is difficult. Therefore, this attack will instead affect the toaster owner, who is now "hacked." The corresponding attack step is simply added as such

> asset Owner {

> | hacked

> }

This sequence of events is facilitated by this association defined earlier

> Owner [owner] 1 <-- owns --> * [toasters] Toaster

The final MAL specification, with the previous theft included, should now have the the following structure

> #id: "org.toasterLang"

> #version: "1.0.0"

> category System {

> asset Toaster{

> | attemptToastRemoval

> -> toastRemoval

> E ifToastExists

> <- slots.toast

> -> toastRemoval

> & toastRemoval

> -> slots.toast.remove

> | attemptToasting

> -> unauthorizedToasting

> !E ifToastNotExists

> <- slots.toast

> -> unauthorizedToasting

> & unauthorizedToasting

> -> owner.hacked

>}

> asset Slot {}

> asset Toast {

> | remove

>}

> asset Owner {

> | hacked

> }

>}

> associations{
   
>  Owner [owner] 1 <-- owns --> * [toasters] Toaster 
    
>  Toaster [toaster] 1 <-- hosts --> * [slots] Slot 
    
>  Slot [slot] 1 <-- toasts --> 0..1 [toast] Toast
    
>}


# Instantiation and simulation

## Preamble

Basic tests are conducted using Java and JUnit. Source code files are found in "\*/FDD_toasterLang/src/test/java/org/toasterLang/". The following tests will be written in the file "TestToaster.java" in that location. Compiling and executing tests is done by typing

> mvn test

Into a terminal at the project root location, i.e., "\*/FDD_toasterLang/".

### The test file

Test files require some setting up before test cases can be written and executed. The following lines should be added to the file

> package org.toasterLang.test;

> import core.Attacker;

> import core.AttackStep;

> import core.Asset;

> import core.Defense;

> import org.junit.jupiter.api.Test;

> import org.junit.jupiter.api.AfterEach;

> public class TestToaster {

> @AfterEach

> public void deleteModel(){

> Asset.allAssets.clear();

> AttackStep.allAttacksteps.clear();

> Defense.allDefenses.clear();

>}

>}


## The waterfall test structure

A single test case can be written in the following structure

1. Create assets
2. Set up associations
3. Prepare attacker
4. Test assertions

This style is very simple and straightforward but inefficient in the long term. However, it works very well for basic demonstrations.

### Testing toast theft

The first test will be to steal toast from an empty toaster. Start by setting up the test case in-between the TestToaster class and the AfterEach section

> public class TestToaster {

> @Test

> public void testToastTheftNoToast() {

> System.out.println("Toast theft attempt, empty toaster");

>}

> @AfterEach

> public void deleteModel(){

> Asset.allAssets.clear();

> AttackStep.allAttacksteps.clear();

> Defense.allDefenses.clear();

>}

>}

Now, create the Toaster and Slot assets. Omit the Toast asset since the toaster should be empty

> public void testToastTheftNoToast() {

> System.out.println("Toast theft attempt, empty toaster");

> Toaster toaster = new Toaster("toaster");

> Slot slot = new Slot("slot");

>}

Connect the Toaster and the Slot as such

> toaster.AddSlot(slot);

The attacker and their starting point is created via these lines

> Attacker attacker = new Attacker();

> attacker.addAttackPoint(toaster.attemptToastRemoval);

> attacker.attack();

What happened was that the Attacker was created, placed at toaster.attemptToastRemoval, and the attack was finally simulated under "attacker.attack();".

After the attack has been simulated, results can be checked for individual attack steps in the model. First, the attempted toast removal must be true according to

> toaster.attemptToastRemoval.assertCompromisedInstantaneously();

However, the toast theft must have failed since the toaster was empty. This is checked with

> toaster.toastRemoval.assertUncompromised();

Similarly, the unauthorized toasting will also have failed. This is because the attacker was not set up to attempt an unauthorized toasting. This can be checked with 

> toaster.unauthorizedToasting.assertUncompromised();

The full test case should look like

> @Test

> public void testToastTheftNoToast() {

> System.out.println("Toast theft attempt, empty toaster");

> Toaster toaster = new Toaster("toaster");

> Slot slot = new Slot("slot");

> toaster.AddSlot(slot);

> Attacker attacker = new Attacker();

> attacker.addAttackPoint(toaster.attemptToastRemoval);

> attacker.attack();

> toaster.attemptToastRemoval.assertCompromisedInstantaneously();

> toaster.toastRemoval.assertUncompromised();

> toaster.unauthorizedToasting.assertUncompromised();

>}


The newly written test can be run from the project root folder by passing

> mvn test

The attempted toast removal should succeed but the rest should fail. 

The test case for a fully successful toast theft is almost identical


> @Test

> public void testToastTheft() {

> System.out.println("Toast theft");

> Toaster toaster = new Toaster("toaster");

> Slot slot = new Slot("slot");

> Toast toast = new Toast("toast");

> toaster.AddSlot(slot);

> slot.addToast(toast);

> Attacker attacker = new Attacker();

> attacker.addAttackPoint(toaster.attemptToastRemoval);

> attacker.attack();

> toaster.attemptToastRemoval.assertCompromisedInstantaneously();

> toaster.toastRemoval.assertCompromisedInstantaneously();

> toaster.unauthorizedToasting.assertUncompromised();

>}

The only differences are the lines

> Toast toast = new Toast("toast");

> slot.addToast(toast);

> toaster.toastRemoval.assertCompromisedInstantaneously();

A Toast asset has been added, connected to a Slot, and now toastRemoval should succeed. The full test file should now look resemble this

> package org.toasterLang.test;

> import core.Attacker;

> import core.AttackStep;

> import core.Asset;

> import core.Defense;

> import org.junit.jupiter.api.Test;

> import org.junit.jupiter.api.AfterEach;

> public class TestToaster {

> @Test

> public void testToastTheftNoToast() {

> System.out.println("Toast theft attempt, empty toaster");

> Toaster toaster = new Toaster("toaster");

> Slot slot = new Slot("slot");

> toaster.AddSlot(slot);

> Attacker attacker = new Attacker();

> attacker.addAttackPoint(toaster.attemptToastRemoval);

> attacker.attack();

> toaster.attemptToastRemoval.assertCompromisedInstantaneously();

> toaster.toastRemoval.assertUncompromised();

> toaster.unauthorizedToasting.assertUncompromised();

>}

> @Test

> public void testToastTheft() {

> System.out.println("Toast theft");

> Toaster toaster = new Toaster("toaster");

> Slot slot = new Slot("slot");

> Toast toast = new Toast("toast");

> toaster.AddSlot(slot);

> slot.addToast(toast);

> Attacker attacker = new Attacker();

> attacker.addAttackPoint(toaster.attemptToastRemoval);

> attacker.attack();

> toaster.attemptToastRemoval.assertCompromisedInstantaneously();

> toaster.toastRemoval.assertCompromisedInstantaneously();

> toaster.unauthorizedToasting.assertUncompromised();

>}

> @AfterEach

> public void deleteModel(){

> Asset.allAssets.clear();

> AttackStep.allAttacksteps.clear();

> Defense.allDefenses.clear();

>}

>}

Testing the unauthorized toasting is just a matter of changing the attacker's starting points and modifying the assertions accordingly. For example

> @Test

> public void testToasting() {

> System.out.println("Unauthorized toasting");

> Toaster toaster = new Toaster("toaster");

> Slot slot = new Slot("slot");

> Toast toast = new Toast("toast");

> toaster.AddSlot(slot);

> slot.addToast(toast);

> Attacker attacker = new Attacker();

> attacker.addAttackPoint(toaster.attemptToasting);

> attacker.attack();

> toaster.attemptToastRemoval.assertUncompromised();

> toaster.attemptToasting.assertCompromisedInstantaneously();

> toaster.toastRemoval.assertUncompromised();

> toaster.unauthorizedToasting.assertCompromisedInstantaneously();

>}

### The curious case of both attacks at once

There are two things to keep in mind when simulating with MAL

1. Simulation models are static. They cannot be updated dynamically at runtime.
2. Time is not simulated.

Given the above, simulating both toast theft and unauthorized toasting might produce unexpected results. Starting with both attacks for a full toaster as such

> @Test

> public void testBoth() {

> System.out.println("Both attacks, full toaster");

> Toaster toaster = new Toaster("toaster");

> Slot slot = new Slot("slot");

> Toast toast = new Toast("toast");

> toaster.AddSlot(slot);

> slot.addToast(toast);

> Attacker attacker = new Attacker();

> attacker.addAttackPoint(toaster.attemptToasting);

> attacker.addAttackPoint(toaster.attemptToastRemoval);

> attacker.attack();

> toaster.attemptToastRemoval.assertCompromisedInstantaneously();

> toaster.attemptToasting.assertCompromisedInstantaneously();

> toaster.toastRemoval.assertCompromisedInstantaneously();

> toaster.unauthorizedToasting.assertUncompromised();

>}

will produce the following

- Both starting points will succeed.
- Stealing toast will succeed.
- Adding toast will fail.

Intuition would suggest that both attacks succeed. That is, toast is first removed then added. However, recall that *MAL does not simulate time*. Therefore, the theft succeeds because the Toast existed. Unauthorized toasting failed because the toaster was full. Retrying the above with an empty toaster, no Toast asset connected, would produce the opposite result.

# Conclusion
As demonstrated, this format could be used for MAL tutorials. More realistically, it could be used to guide someone in reproducing test results rather than constructing a language. However, this format is redundant for our purposes. It is not infeasible that it could be used at some point in time. However, at the time of writing it might be better to provide instructions e.g. via GitHub together with the source files.